In [18]:
import visa
import serial
import numpy as np
from time import time, ctime, sleep
import matplotlib.pyplot as plt
import io
rm = visa.ResourceManager()

In [ ]:
instruments = (rm.list_resources())
print(instruments)

In [ ]:
### Lock-in functions ###

#### Lock-in communication  ####
# This function communicates with the Lock-in via Serial port. Always check the Lock-in USB port before start.
# We send the cmd via write() function. We need to send the command as ASCII language and the '\n' to indicate the end
# of the command, so we use the encode function to translate the input to ASCII.
# We receive the Lock-in output with the readline() function, and save it in the array value. The strip() function will
# remove blank spaces from the output string. Usually, only when you ask the SNAP? to the Lock-in it will return an output.
# The decode() function will return the translation from ASCII.

def ask_lockin(cmd):
    lok = serial.Serial("COM4", baudrate=9600, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS, timeout=3)
    lok.write(str.encode(cmd+'\n'))
    lok.flush()
    value = lok.readline().strip()
    value = value.decode('ascii')
    lok.close()
    if 'SNAP?' in cmd:
        return value
    else:
        return

### Manual Sense ###
# The Sensing() function will manually set the correct voltage range in the Lock-in front panel in order to increase the 
# measurement resolution. We can achieve the same result by using the cmd AGAN, when the time constant is less than 1 second.
# This function will set the maximum resolution (1V), and then will search the right resolution by comparing the 
# front panel output to certain values. When the range is found, we set the final Lock-in resolution.
# This code will only work for output signals bigger than 5 mV, which is the minimum Vrms from the internal signal source
# of the Lock-in.

def Sensing():
    ask_lockin('SENS26')
    temp = ask_lockin('SNAP?9,3,4')
    temp = temp.split(",")
    V_out = float(temp[1])
    if (V_out < 0.0005):
        ask_lockin('SENS17')
        return
    elif (V_out >= 0.0005 and V_out < 0.001):
        ask_lockin('SENS18')
        return
    elif (V_out >= 0.001 and V_out < 0.003):
        ask_lockin('SENS19')
        return
    elif (V_out >= 0.003 and V_out < 0.008):
        ask_lockin('SENS20')
        return
    elif (V_out >= 0.008 and V_out < 0.018):
        ask_lockin('SENS21')
        return
    elif (V_out >= 0.018 and V_out < 0.048):
        ask_lockin('SENS22')
        return
    elif (V_out >= 0.048 and V_out < 0.090):
        ask_lockin('SENS23')
        return
    elif (V_out >= 0.090 and V_out < 0.190):
        ask_lockin('SENS24')
        return
    elif (V_out >= 0.190 and V_out < 0.490):
        ask_lockin('SENS25')
        return
    elif (V_out >= 0.490):
        ask_lockin('SENS26')
        return
    #print(V_out)

### Measurement routine ###
# Inputs: Voltage amplitude (A) in volts and signal frequency (F) in Hz.
# Output: string containing the Lock-in source voltage, output voltage amplitude and phase.
def Measurement():
    Sensing()                     ## Calls the manual sensing function
    temp = ask_lockin('SNAP?3,4') ## Reads the output voltage amplitude and phase
    return(temp)

def Setup(V_amplitude, Input_freq, Sampling_rate):
    ask_lockin('REST')                   ## Clears the memory buffer
    ask_lockin('SLVL'+str(V_amplitude))  ## Set the Lock-in voltage amplitude (V)
    ask_lockin('FREQ'+str(Input_freq))   ## Set the Lock-in frequency (Hz)
    ask_lockin('SRAT'+str(Sampling_rate))## Sampling rate
    ask_lockin('SEND1')                  ## Measures in loop mode
    ask_lockin('STRT')                   ## Starts measurement

In [ ]:
### Arduino functions ###
# Always check the Arduino USB port before start. This function will send movement request to the Arduino, according to the
# following table:
## 1 - Up
## 2 - Down
## 3 - Left
## 4 - Right
## 7 - Return to initial point (left lateral movement)

def ask_Arduino(Mov, D_vel, U_vel, Z, Immersion_time, Washing_time):
    #arduino = serial.Serial('COM9')
    #arduino.readline()
    cmd = str(Mov)+','+str(D_vel)+','+str(U_vel)+','+str(Z)+','+str(Immersion_time)+','+str(Washing_time)+'\n'
    arduino.write(cmd.encode('utf-8'))
    arduino.readline()
    #arduino.close()
    return

### Mono layer deposition routine ###
def mono_layer(Z, Immersion_time, Washing_time):
    ask_Arduino(2, 10, 60, Z, 0, 0)
    sleep(Immersion_time*60)
    ask_Arduino(1, 10, 60, Z, 0, 0)
    #ask_Arduino(4, 10, 60, Z, 0, 0)
    ask_Arduino(2, 10, 60, Z, 0, 0)
    sleep(Washing_time)
    ask_Arduino(1, 10, 60, Z, 0, 0)
    return

In [ ]:
arduino = serial.Serial('COM9')

In [ ]:
#### Positioning routine ####
# Always check the Arduino USB port before start.
# Use this routine to position the IDE on top of the desired polyelectrolyte.
# Use the following dictionary to move the cart left and right, and the IDE up and down. You can only control the vertical
# motion. The lateral displacement is already set on the Arduino code. If you need to calibrate the vertical displacement,
# keep in mind that the Arduino code will always move it 56 mm in the deposition procedure.
## 1 - Up
## 2 - Down
## 3 - Left
## 4 - Right
## 7 - Return to initial point (left lateral movement)
Mov = 1 # movement direction
Z = 56 # vertical displacement (mm)
ask_Arduino(Mov, 80, 80, Z, 0, 0)


In [ ]:
### Mono layer analysis ###
# This routine will deposite one monolayer. It should start at the desired polyelectrolyte, and then it will proceed to
# wash procedure. You have to enter the immersion, washing and drying times.
# After this, we proceed to measurements of the film capacitance.

## Archive name ##
archive_name = 'PEI_01'

## Deposition parameters ##
Z = 10 # vertical displacement (mm)
Immersion_time = 0.1 # min
Washing_time = 0 # sec
Drying_time = 1 # min

## Measurement parameters ##
V_amplitude = 1 # V
Frequency = 1000 # Hz
Sampling_rate = 10 # 64 Hz - manual (pg5-13) table with allowed sampling rates
ref = 0.5 # accepted average mean distance of measurements taken after drying process

## Mono layer deposition routine ##
mono_layer(Z, Immersion_time, Washing_time)

Setup(V_amplitude, Frequency, Sampling_rate) # Setup in the Lock-in the measurement parameters

## Capacitance measurements during drying time ##
t_i = time()
t1 = t_i
t2 = t1 + (Drying_time*60)

freq = np.empty(shape=[0])
R = np.empty(shape=[0])
Theta = np.empty(shape=[0])
Time = np.empty(shape=[0])

try:
    lok = serial.Serial("COM4", baudrate=9600, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS, timeout=3)
    cmd = str.encode('SNAP?3,4\n')

    while t2 >= t1:
        #temp = Measurement()
        lok.write(cmd)
        temp = lok.readline().strip()
        lok.flush()
        temp = temp.decode('ascii')
        temp = temp.split(",")
        #freq = np.append(freq, [float(temp[0])], axis=0)
        R = np.append(R, [float(temp[0])], axis=0)
        Theta = np.append(Theta, [float(temp[1])], axis=0)
        Time = np.append(Time, [round((t1 - t_i),1)], axis=0)
        t1 = time()

    while True:
        t = time() - t_i
        #temp = Measurement()
        lok.write(cmd)
        temp = lok.readline().strip()
        lok.flush()
        temp = temp.decode('ascii')
        temp = temp.split(",")
        #freq = np.append(freq, [float(temp[0])], axis=0)
        R = np.append(R, [float(temp[0])], axis=0)
        Theta = np.append(Theta, [float(temp[1])], axis=0)
        Time = np.append(Time, [round(t,1)], axis=0)
        if len(Time) > 10:
            mean = np.mean(R[-10:])
            amd = np.mean(np.absolute(R[-10:])-mean)
            if amd < ref:
                break
finally:
    lok.close()

arq = open(archive_name+'.dat', 'w')
arq.write('Time (s)\t Voltage Amplitude (V)\t Phase \n')
for i in range(len(Time)):
    arq.write(str(Time[i]) + '\t')
    #arq.write(str(freq[i]) + '\t')
    arq.write(str(R[i]) + '\t')
    arq.write(str(Theta[i]) + '\n')
arq.close()

In [34]:
Setup(1, 1000, 13) # Setup in the Lock-in the measurement parameters

[-0.08723751 -0.59011529 -0.42972505  0.18763283  1.15132505  1.54994611
  1.51784918 -1.09822292  1.79548665  0.40823567 -0.96439257  0.48356317
 -0.5099794   0.6078387  -0.01483535  1.3268447   1.72879015 -1.43270538
 -0.01138176 -0.38544264  1.12411897 -1.07192109  0.68902252 -0.85365499
  0.24086577 -1.33335389 -0.9508139  -0.60146868  1.13385503]


In [ ]:
lok = serial.Serial("COM4", baudrate=9600, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS, timeout=3)
t1 = time()
t2 = t1 + 20
cmd = str.encode('SNAP?3,4\n')
#temp = []

try:
    while t2 >= t1:
        lok.write(cmd)
        temp += lok.readline().strip()
        lok.flush()
        t1 = time()

    temp = temp.decode('ascii')
    temp = temp.split(',')
    print(temp)

finally:
    lok.close()

In [ ]:
lok = serial.Serial("COM4", baudrate=9600, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS, timeout=3)
cmd = str.encode('TRCA?1,0,16000\n')
lok.write(cmd)
temp = lok.readlines().strip()
temp = temp.decode('ascii')
lok.flush()
temp.split(',')
print (temp)
lok.close()